In [1]:
import pandas as pd
import numpy as np
from Lib import Algen_lib as lib

# Load Data

In [2]:
#Data Latih
dfA = pd.read_excel('data/sms_clean_norm_+_new_data_3.xlsx')
df =dfA[100:] 
corpus = df.sms.tolist()
label = df.label.tolist()
len(dfA)

1143

### Pembagian Data Uji

In [3]:
#Data Uji
df =dfA[:100] 
corpusU = df.sms.tolist()
labelU = df.label.tolist()

In [4]:
# df =dfA[100:] 
corpusA = dfA.sms.tolist()
labelA = dfA.label.tolist()

# Ekstraksi Fitur

In [5]:
fitur = lib.feature_extraction(corpus)

# Seleksi Fitur Menggunaka Algoritma Genetika

In [6]:
jumlah_populasi = 200
jumlah_point = 1000 #crossover
prob_mutasi = 0.2 #probabilitas permutasi

K=12

metode = "tfidf"
alpha = 1

target = 0.99 #target akurasi

# fitur = lib.feature_extraction(corpus)
panjang_fitur = len(fitur)

features_1_bin = [1 for i in range(panjang_fitur)]
pop = lib.create_population(jumlah_populasi-1, panjang_fitur)
pop.append(features_1_bin)

fns_1 = lib.fitness_kf(corpusA, labelA, features_1_bin, fitur, alpha = alpha, metode = metode, K=K)
print("fitness all features:",fns_1)

fitnes_pop = list()
temp_features_fitness = list()
for features_bin in pop:
    fns = lib.fitness_kf(corpusA, labelA, features_bin, fitur, alpha = alpha, metode = metode, K=K)
    fitnes_pop.append(fns)
    temp_features_fitness.append([fns, features_bin])

pop_used = dict()
popp_0 = list(pop)
for p, fit in zip(popp_0, fitnes_pop):
    bin_str = lib.int_to_str(p) 
    pop_used.update({bin_str:fit})

# print("")
    
# print(pop[0])

best_fitur_list = list()
best_fitness_list = list()
best_generasi_list = list()

# save_aja = list()

gen=0
while True:
    gen+=1
    print("-"*25)
    print("Generasi ke",gen , "[Terbaik]")
    print("   "+"________________________________________")
    print("     Nilai Fitness       |fitur|ALL  |Generasi")
    rw = lib.get_roulette_wheel_(fitnes_pop, inc=jumlah_populasi)
    parents = lib.select_parents(rw)

    best_fitur_list_per_generasi = list()
    best_fitness_list_per_generasi = list()
    for p in parents:
        best_generasi_list.append(gen)
        family_fitness=list()
        family_fitur=list()

        mama_index = p[0]
        papa_index = p[1]
        
        m4ms = fitnes_pop[mama_index]
        family_fitness.append(m4ms)
        m4mr = list(pop[mama_index])
        family_fitur.append(m4mr)
        
        p4ps = fitnes_pop[papa_index]
        family_fitness.append(p4ps)
        p4pr = list(pop[papa_index])
        family_fitur.append(p4pr)
        
        #crossover dan mutasi
        bin_mama = list(pop[mama_index])
        bin_papa = list(pop[papa_index])
        anak_binary = lib.crossover(bin_mama, bin_papa, panjang_fitur,  jumlah_titik = jumlah_point, prob_mutasi = prob_mutasi)
        
        #mencari fitness untuk child
        a1 = anak_binary[0]
        a2 = anak_binary[1]
#         print(a2)
        
        if sum(a1)<=0:
            a1[-1]=1
            a1[-2]=1
        if sum(a2)<=0:
            a2[-1]=1
            a2[-2]=1

        str_a1 = lib.int_to_str(a1)
        if str_a1 in pop_used:
            family_fitur.append(a1)
            family_fitness.append(pop_used[str_a1])  
        else:
            family_fitur.append(a1)
            fitness_anak_1 = lib.fitness_kf(corpusA, labelA, a1, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_1)
            pop_used.update({str_a1:fitness_anak_1})
            
        str_a2 = lib.int_to_str(a2)
        if str_a2 in pop_used:
            family_fitur.append(a2)
            family_fitness.append(pop_used[str_a2])
        else:
            family_fitur.append(a2)
            fitness_anak_2 = lib.fitness_kf(corpusA, labelA, a2, fitur, alpha = alpha, metode = metode, K=K)
            family_fitness.append(fitness_anak_2)
            pop_used.update({str_a2:fitness_anak_2})

        
        #mencari fitness terbaik untuk satu keluarga
        best_family_fitness = max(family_fitness)
        bf_index = family_fitness.index(best_family_fitness)
        best_family_fitur = family_fitur[bf_index]

        best_fitness_list.append(best_family_fitness)
        best_fitur_list.append(best_family_fitur)

        best_fitness_list_per_generasi.append(best_family_fitness)
        best_fitur_list_per_generasi.append(best_family_fitur)

#         print("   ",best_family_fitness)
        print("   ",lib.tampil_finess(best_family_fitness),"|",str(sum(best_family_fitur))+"|"+str(len(best_family_fitur)),"|",gen)

    pop = list(best_fitur_list_per_generasi)
#     pop.append(features_1_bin)
    fitnes_pop = list(best_fitness_list_per_generasi)
#     fitnes_pop.append(fns_1)

    if len(pop)<2 or max(best_fitness_list)>=target:
        print("="*70)
        print("Best",max(best_fitness_list))
#         print("Generasi ke-",best_generasi_list[best_fitness_list.index(max(best_fitness_list))])
        good_fitur = best_fitur_list[best_fitness_list.index(max(best_fitness_list))]
        good_fitur_2 = best_fitur_list[-1]
        best_score_2 = best_fitness_list[-1]
        best_fitness_list.index(max(best_fitness_list))
        print('jumlah fitur     ', sum(good_fitur))
        print('jumlah fitur asli', panjang_fitur)
        print('good_fitur')
        break

fitness all features: 0.9352796052631581
-------------------------
Generasi ke 1 [Terbaik]
   ________________________________________
     Nilai Fitness       |fitur|ALL  |Generasi
    0.9115862573099416   | 2071|4144 | 1
    0.9107456140350876   | 2050|4144 | 1
    0.914281798245614    | 2101|4144 | 1
    0.9177448830409355   | 2036|4144 | 1
    0.9177814327485381   | 2096|4144 | 1
    0.9177540204678363   | 2085|4144 | 1
    0.9221034356725145   | 2108|4144 | 1
    0.9055555555555554   | 2073|4144 | 1
    0.9133954678362572   | 2042|4144 | 1
    0.9107638888888888   | 2074|4144 | 1
    0.918658625730994    | 2091|4144 | 1
    0.9168676900584792   | 2095|4144 | 1
    0.923044590643275    | 2014|4144 | 1
    0.9116410818713451   | 2063|4144 | 1
    0.9195266812865497   | 2039|4144 | 1
    0.9142543859649122   | 2059|4144 | 1
    0.9195266812865496   | 2077|4144 | 1
    0.9177174707602339   | 2080|4144 | 1
    0.9081323099415205   | 2093|4144 | 1
    0.916858552631579    | 2058|4144 | 

    0.9204038742690058   | 2050|4144 | 4
    0.923044590643275    | 2014|4144 | 4
-------------------------
Generasi ke 5 [Terbaik]
   ________________________________________
     Nilai Fitness       |fitur|ALL  |Generasi
    0.9212902046783626   | 2103|4144 | 5
    0.926498538011696    | 2097|4144 | 5
    0.929970760233918    | 2047|4144 | 5
    0.9264802631578948   | 2063|4144 | 5
    0.9352796052631581   | 4144|4144 | 5
    0.923044590643275    | 2014|4144 | 5
    0.9282711988304092   | 2097|4144 | 5
-------------------------
Generasi ke 6 [Terbaik]
   ________________________________________
     Nilai Fitness       |fitur|ALL  |Generasi
    0.9352796052631581   | 4144|4144 | 6
    0.929970760233918    | 2047|4144 | 6
    0.9282711988304092   | 2097|4144 | 6
    0.9264802631578948   | 2063|4144 | 6
-------------------------
Generasi ke 7 [Terbaik]
   ________________________________________
     Nilai Fitness       |fitur|ALL  |Generasi
    0.9352796052631581   | 4144|4144 | 7
   

In [9]:
#Testing
hasil_seleksi_fitur = good_fitur
lib.fitness_(corpus,label, corpusU,labelU, good_fitur_2, fitur, alpha = alpha, metode = metode)

0.92

In [8]:
fitur_pilihan = np.array(fitur)[lib.get_index(good_fitur)]
print(len(fitur_pilihan))
fitur_pilihan

4144


array(['0', '00', '000', ..., 'zalora', 'zarkasi', 'zona'], dtype='<U26')

In [62]:
# skor_all